In [1]:
from os import listdir
import jsonlines
import itertools
import nltk
from nltk.corpus import stopwords
import pymorphy2 as pm
import networkx as nx
import re
from bisect import bisect_left
#from joblib import Parallel, delayed
from collections import Counter
from tqdm import trange
from tqdm import tqdm
from math import log
from random import shuffle
from sys import exit
import numpy as np
import json
import codecs
import random

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.neighbors import NearestCentroid
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import load_npz
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
#nltk.download()
tknzr = nltk.TweetTokenizer()

In [2]:
ok_set = set()
with open("sources/accepted_categories.txt", mode="r", encoding="utf-8") as inp:
    for line in inp:
        line = line[:-1]
        ok_set.add(line)

ok = sorted(ok_set)

categories_dict = {}
with open("sources/article_cat.json", mode="r") as input:
    categories_dict = json.loads(input.read())
    
category_article = {}
with open("sources/cat_article.json", mode="r") as input:
    category_article = json.loads(input.read())
    
X = load_npz("sources/text_tfidf.npz")
    
texts = {item['id']: item['text'] for item in jsonlines.open('sources/normalized_texts.jl', 'r')}

In [3]:
ids = sorted([id for id in texts.keys()])

In [4]:
vectorizer = TfidfVectorizer(min_df=5, max_df=0.98)
text_tfidf = vectorizer.fit_transform([text for (id, text) in sorted(texts.items())])
#svd = TruncatedSVD(n_components=500, random_state=27)
#X_svd = svd.fit_transform(text_tfidf)
X = text_tfidf

In [4]:
k_nearest = 5
'''class_centroids = {}
with open("sources/centroids.json", "r") as input:
    class_centroids = json.loads(input.read())
'''
class_centroids = load_npz("sources/sparce_centroids_nosvd.npz")

kn_clust = NearestNeighbors(n_neighbors=k_nearest)
#kn_clust.fit([centroid for (cat, centroid) in sorted(class_centroids.items())])
kn_clust.fit(class_centroids)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [5]:
class_centroids.shape

(14697, 121303)

In [6]:
X.shape

(96794, 121303)

In [7]:
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=27)
clf = LogisticRegression(C=1.0)
X_train = []
X_right = []
X_left = []
y_train = []
rows = []

train_index, test_index = 0, 0
for item in kf.split(ids):
    
    score = 0
    train_index, test_index = item[0], item[1]
    
    print("TRAIN:", train_index, "TEST:", test_index)
    print(len(train_index), len(test_index))
    
    X_left[:] = []
    for i in trange(len(train_index)):
        for j in range(k_nearest):
            X_left.append(X[train_index[i]])
            
    X_right[:] = []
    y_train[:] = []
    rows[:] = []
    print("Train!")
    for i, index in enumerate(tqdm(train_index)):
        cats = categories_dict[ids[index]]
        curr_x = X[index]
        neighbors = kn_clust.kneighbors(curr_x, n_neighbors=k_nearest, return_distance=False)
        neighbors = [ok[c] for c in neighbors[0]]
        common = set(neighbors).intersection(set(cats))
        for neigh in neighbors:
            rows.append(ok.index(neigh))
            y_train.append(1 if neigh in common else 0)
    
    X_right = X[rows]
    X_train = hstack([X_left, X_right])
    clf.fit(np.array(X_train), np.array(y_train))
    
    print("Test!")
    for index in tqdm(test_index):
        result = ids[index]
        test_svd = X[index]
        y_true = np.array(categories_dict[result])
        X_local = []
        local_skipped = []
        prediction = []
        for i, y in enumerate(y_true):
            if class_centroids.get(y) is not None:
                row = ok.index(neigh)
                X_local.append(hstack([test_svd, class_centroids[row].toarray()[0]]))
            else:
                local_skipped.append((i, y))
        if len(X_local) > 0:
            prediction = clf.predict(X_local)
        for item in local_skipped:
            prediction.insert(item[0], 1)
            
        score += sum(prediction) / len(y_true)
    
    print(score / len(test_index))   

TRAIN: [    0     1     2 ... 96791 96792 96793] TEST: [    4     7    27 ... 96770 96779 96789]
77435 19359


100%|██████████████████████████████████████████████████████████████████████████| 77435/77435 [00:29<00:00, 2664.33it/s]


Train!


  1%|▍                                                                          | 475/77435 [08:32<23:04:07,  1.08s/it]

KeyboardInterrupt: 